<a href="https://colab.research.google.com/github/chobocoding/deeplearning/blob/main/PPG(Conv1d).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/IT융합/ppg')

In [ ]:
training_subject_list = [2,3,4,5,6,7,8,9,10,11,13,14]
validation_subject_list = [15,16]
test_subject_list = [17]

In [ ]:
class PPG_Dataset(Dataset):
    def __init__(self, subject_list):
        super(PPG_Dataset, self).__init__()

        ppg_list = []
        for sbj_num in subject_list:
            file_name = 'subject_' + str(sbj_num) + '.npy'
            ppg = np.load(file_name)
            ppg_list.append(ppg)

        self.ppg_merge = np.concatenate(ppg_list, axis=0)
                        
    def __getitem__(self, index):
        ppg_sample = self.ppg_merge[index, :7680]
        ppg_tensor = torch.tensor(ppg_sample[np.newaxis, :]).float()

        label = self.ppg_merge[index, 7680]
        label_tensor = torch.tensor(label.astype(np.int64))

        return ppg_tensor, label_tensor
    
    def __len__(self):
        return self.ppg_merge.shape[0]
        

In [ ]:
train_dataset = PPG_Dataset(training_subject_list)
train_loader = DataLoader(dataset=train_dataset, batch_size=120, shuffle=True)

In [ ]:
class Net(nn.Module):
    def __init__(self, base_dim, num_classes=2):
        super(Net, self).__init__()
        self.feature = nn.Sequential(
            conv_2_block(1, base_dim),
            conv_2_block(base_dim, 2*base_dim),
            conv_3_block(2*base_dim, 8*base_dim),
            conv_3_block(8*base_dim, 16*base_dim),
            conv_3_block(16*base_dim, 16*base_dim),
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(16*base_dim * 30, base_dim * 5),
            nn.ReLU(True),
            nn.Linear(base_dim * 5, base_dim),
            nn.ReLU(True),
            nn.Linear(base_dim, 20),
            nn.ReLU(True),
            nn.Linear(20, num_classes)
    )
    
    def forward(self, x):
        x = self.feature(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x        
    
        return x

In [ ]:
net = Net(base_dim=8).to('cuda:0')
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)

In [ ]:
for epoch in range(25):
    loss_val = 0
    for itr, data in enumerate(train_loader):
        optimizer.zero_grad()
        
        ppg, label = data
        ppg = ppg.to('cuda:0')
        label = label.to('cuda:0')
        
        pred = net(ppg)
        loss = loss_function(pred, label)
        
        loss.backward()
        optimizer.step()
        
        loss_val += loss.item()
        
    print("Epoch:", epoch+1, "  , Loss: ", loss_val)

Epoch: 1   , Loss:  3.645359456539154
Epoch: 2   , Loss:  3.5260998606681824
Epoch: 3   , Loss:  3.3838892579078674
Epoch: 4   , Loss:  3.1695640683174133
Epoch: 5   , Loss:  2.7737025022506714
Epoch: 6   , Loss:  2.288233071565628
Epoch: 7   , Loss:  1.978663295507431
Epoch: 8   , Loss:  1.7991059124469757
Epoch: 9   , Loss:  1.633703961968422
Epoch: 10   , Loss:  1.5560382902622223
Epoch: 11   , Loss:  1.5045807361602783
Epoch: 12   , Loss:  1.4268393069505692
Epoch: 13   , Loss:  1.374856412410736
Epoch: 14   , Loss:  1.2972102910280228
Epoch: 15   , Loss:  1.2112571448087692
Epoch: 16   , Loss:  1.093684732913971
Epoch: 17   , Loss:  0.9770910888910294
Epoch: 18   , Loss:  0.8337254598736763
Epoch: 19   , Loss:  0.6724540889263153
Epoch: 20   , Loss:  0.5143420621752739
Epoch: 21   , Loss:  0.37867724150419235
Epoch: 22   , Loss:  0.28486817702651024
Epoch: 23   , Loss:  0.20569687895476818
Epoch: 24   , Loss:  0.14452447555959225
Epoch: 25   , Loss:  0.10773770697414875


In [ ]:
val_dataset = PPG_Dataset(validation_subject_list)
val_loader = DataLoader(dataset=val_dataset, batch_size=100, shuffle=True)

In [ ]:
pred_list = []
label_list = []
for itr, data in enumerate(val_loader):
    ppg, label = data
    ppg = ppg.to('cuda:0')
    label = label.to('cuda:0')    
    pred_test = net(ppg)

    pred_category = torch.argmax(pred_test, dim=1)
    
    pred_list = pred_list + list(pred_category)
    label_list = label_list + list(label)
    
accu = np.mean( np.array(pred_list)==np.array(label_list) )
print("Validation accuracy: ", accu)

Validation accuracy:  0.9754098360655737


In [ ]:
torch.save(net, "model_97.pth")

In [ ]:
net_load = torch.load("model.pth")

In [ ]:
test_dataset = PPG_Dataset(test_subject_list)
test_loader = DataLoader(dataset=test_dataset, batch_size=100, shuffle=True)

In [ ]:
pred_list = []
label_list = []
for itr, data in enumerate(test_loader):
    ppg, label = data
    
    pred_test = net_load(ppg)
    pred_category = torch.argmax(pred_test, dim=1)
    
    pred_list = pred_list + list(pred_category)
    label_list = label_list + list(label)
    
accu = np.mean( np.array(pred_list)==np.array(label_list) )
print("Test accuracy: ", accu)

Test accuracy:  0.9016393442622951
